<a href="https://colab.research.google.com/github/sumaiya008/CCNY-DSE-Capstone-Project-Segmenting-Coral-Branch-tips/blob/main/notebooks/1.4_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install mrcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mrcnn: filename=mrcnn-0.2-py3-none-any.whl size=54907 sha256=7e9f89d14c67f7b9ca4816d531222a7566196df532cf09278a09e6aaa9fb02bf
  Stored in directory: /root/.cache/pip/wheels/f6/66/35/4956a489776a1db0ee24e0c33a9088c1b5d68eb5dc2654f0b2
Successfully built mrcnn


In [5]:
import os
import numpy as np
import random
import pickle
import skimage
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

from mrcnn.utils import Dataset
from mrcnn import visualize
import cv2
import xml.etree.ElementTree as ET

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
class CoralDataset(Dataset):
    def load_dataset(self, dataset_dir):
        # Add classes. In your case, you have 2 coral classes.
        self.add_class("dataset", 1, "APAL")
        self.add_class("dataset", 2, "Pseododiploria")

        # Define data locations
        images_dir = os.path.join(dataset_dir, 'image')
        annotations_dir = os.path.join(dataset_dir, 'annotation')

        # List all files in the directory
        for filename in os.listdir(images_dir):
            if filename.endswith(".jpg"):
                image_id = os.path.splitext(filename)[0]
                img_path = os.path.join(images_dir, filename)
                ann_path = os.path.join(annotations_dir, image_id + ".xml")
                self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    def load_mask(self, image_id):
        # Extract annotation data from the XML file
        info = self.image_info[image_id]
        annotation_path = info['annotation']
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        masks = []

        # Create masks based on annotation data
        for obj in root.findall(".//object"):
            class_name = obj.find("name").text
            if class_name == "coral_type1":
                class_id = 1
            elif class_name == "coral_type2":
                class_id = 2
            else:
                class_id = 0  # Background

            mask = self.extract_mask_from_xml(obj)  # Implement a function to convert XML data to a binary mask
            masks.append({
                'class_id': class_id,
                'mask': mask,
            })

        masks = [mask['mask'] for mask in masks]
        masks = skimage.morphology.binary_dilation(masks, np.ones((8, 8)))

        # Convert masks list to a stack of binary masks
        masks = np.stack(masks, axis=-1)
        return masks, np.asarray([mask['class_id'] for mask in masks], dtype=np.int32)

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def extract_mask_from_xml(self, obj):
        # Implement this function to convert annotation data in XML to a binary mask
        # You'll need to parse the object's bounding box coordinates and create a binary mask.

        # Example pseudocode:
        # 1. Get the bounding box coordinates from obj (xmin, ymin, xmax, ymax)
        # 2. Create an empty binary mask with the same shape as the image
        # 3. Set the pixels within the bounding box to 1, everything else to 0
        # 4. Return the binary mask

        # For parsing XML data, you can use the ElementTree library (imported as ET)
        pass

In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mrcnn import visualize

# Create an instance of your dataset
dataset = CoralDataset()

# Load the dataset (provide the path to your dataset directory)
dataset.load_dataset('/content/drive/My Drive/Capstone/Coral_images')

# Prepare the dataset (this step is necessary before you can use the dataset)
dataset.prepare()

# Load and visualize an example image with masks
image_id = 0  # Choose an image ID (you can change this to any valid image ID)
image = dataset.load_image(image_id)
masks, class_ids = dataset.load_mask(image_id)

# Visualize the image
plt.imshow(image)
plt.axis('off')
plt.title('Example Coral Image')
plt.show()

# Visualize the masks
class_names = dataset.class_names
visualize.display_instances(image, masks, class_ids, class_names, figsize=(8, 8))


AttributeError: ignored

In [ ]:
# Load the augmented data from the file
data = np.load('/content/drive/My Drive/Capstone/Pickled/coral_augmented_combined.npz')

# Extract the X_train, y_train, X_test, and y_test arrays from the data
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

In [ ]:
# Check the shape of the loaded data
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (4314, 224, 224, 3)
y_train shape: (4314,)
X_test shape: (1080, 224, 224, 3)
y_test shape: (1080,)
